In [1]:
!date

Wed Jun 22 14:09:48 PDT 2022


___

# DESeq2 - Differential gene expression analysis based on the negative binomial distribution
Activate environment:  
conda activate diffexpr

Cite:  
https://github.com/wckdouglas/diffexpr  
https://bioconductor.org/packages/release/bioc/html/DESeq2.html  

DESeq2 manual:  
http://www.bioconductor.org/packages/release/bioc/manuals/DESeq2/man/DESeq2.pdf

In [2]:
%config InlineBackend.figure_format = 'retina'
%load_ext blackcellmagic

In [3]:
import sys
import anndata
# import scvi
import pandas as pd
import scanpy as sc
import numpy as np
from scipy import stats
import gget

from diffexpr.py_deseq import py_DESeq2

from upsetplot import UpSet

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import cm

import glob

sc.set_figure_params(figsize=(6, 6), frameon=False)
sc.settings.n_jobs=2

In [4]:
# set random seed
np.random.seed(926)

In [5]:
def nd(arr):
    """
    Function to transform numpy matrix to nd array.
    """
    return np.asarray(arr).reshape(-1)

___

# Load AnnData object

In [6]:
adata = anndata.read_h5ad("../../finchseq_data/all_celltype.h5ad")
adata

AnnData object with n_obs × n_vars = 35804 × 22151
    obs: 'species', 'batch', 'n_counts_processed', 'batch_index', 'n_counts_raw', 'leiden', 'celltype', 'connectivity'
    var: 'gene_name', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'batch_colors', 'celltype_colors', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

Define masks to separate control and experiment datasets:

In [7]:
control_mask = np.logical_or(adata.obs["batch"]=="control1", adata.obs["batch"]=="control2")
experiment_mask = np.logical_or(adata.obs["batch"]=="experiment1", adata.obs["batch"]=="experiment2")

Add new obs column to separate between control and experiment in general, without separating between batches:

In [8]:
adata.obs["batch_g"] = ""

adata.obs.loc[control_mask, "batch_g"] = "control"
adata.obs.loc[experiment_mask, "batch_g"] = "experiment"

# Create columns containing general celltype assignment - ignoring cluster separation
adata.obs['celltype_g'] = adata.obs['celltype'].str.replace('\d+', '')

adata.obs.head()

,species,batch,n_counts_processed,batch_index,n_counts_raw,leiden,celltype,connectivity,batch_g,celltype_g
barcode,,,,,,,,,,
AAACCCAAGCACTGGA-0,zebrafinch,control1,483.000000,0,483.000000,13,mural cells 2,0,control,mural cells
AAACCCAAGCGTCAAG-0,zebrafinch,control1,946.000061,0,946.000061,3,microglia 1,1,control,microglia
AAACCCAAGGTCACAG-0,zebrafinch,control1,1068.000000,0,1068.000000,1,GABAergic neurons 1,2,control,GABAergic neurons
AAACCCAAGTCATTGC-0,zebrafinch,control1,1407.000000,0,1407.000000,10,astrocytes 2,3,control,astrocytes
AAACCCAAGTGCTACT-0,zebrafinch,control1,1060.000000,0,1060.000000,1,GABAergic neurons 1,2,control,GABAergic neurons


___

# DESeq2 on mean gene expression per sample

In [123]:
%%time

for cluster in adata.obs.celltype.values.unique():
    # Create masks to obtain only retain cells from control or experiment, and one cluster
    celltype_mask_control1 = np.char.startswith(nd(adata.obs.celltype.values).astype(str), cluster) & [adata.obs["batch"]=="control1"]
    celltype_mask_control2 = np.char.startswith(nd(adata.obs.celltype.values).astype(str), cluster) & [adata.obs["batch"]=="control2"]
    celltype_mask_experiment1 = np.char.startswith(nd(adata.obs.celltype.values).astype(str), cluster) & [adata.obs["batch"]=="experiment1"]
    celltype_mask_experiment2 = np.char.startswith(nd(adata.obs.celltype.values).astype(str), cluster) & [adata.obs["batch"]=="experiment2"]

    # Apply masks to raw data
    control_data1 = adata.raw[celltype_mask_control1]
    control_data2 = adata.raw[celltype_mask_control2]
    experiment_data1 = adata.raw[celltype_mask_experiment1]
    experiment_data2 = adata.raw[celltype_mask_experiment2]

    # Filter for only highly variable genes
    control_data1 = control_data1[:,adata.var['highly_variable']]
    control_data2 = control_data2[:,adata.var['highly_variable']]
    experiment_data1 = experiment_data1[:,adata.var['highly_variable']]
    experiment_data2 = experiment_data2[:,adata.var['highly_variable']]

    ## Build count matrix
    count_matrix = pd.DataFrame()
    count_matrix["gene"] = adata.raw.var[adata.var['highly_variable']==True].index.values

    # Get mean gene expression per condition for each gene (rounded to integer)
    count_matrix["C_1"] = np.around(control_data1.X.mean(axis=0).A1).astype(int)
    count_matrix["C_2"] = np.around(control_data2.X.mean(axis=0).A1).astype(int)
    count_matrix["E_1"] = np.around(experiment_data1.X.mean(axis=0).A1).astype(int)
    count_matrix["E_2"] = np.around(experiment_data2.X.mean(axis=0).A1).astype(int)

    ## Build design matrix
    design_matrix = pd.DataFrame()
    design_matrix["samplename"] = count_matrix.columns.values[1:]
    design_matrix = design_matrix.assign(
        sample=lambda d: d.samplename.str.extract("([CE])_", expand=False)
    )
    design_matrix["replicate"] = design_matrix["samplename"].str.extractall("(\d+)").values
    design_matrix.index = design_matrix.samplename


    # Run DESeq2
    try:
        dds = py_DESeq2(
            count_matrix=count_matrix,
            design_matrix=design_matrix,
            design_formula="~ replicate + sample",
            gene_column="gene",
        )
        dds.run_deseq(
            test="LRT",                            # Likelihood ratio test on the difference in deviance between a full and reduced model formula (defined by nbinomLRT)
            reduced="~ replicate",                 # Reduced model without condition as variable
            # fitType="parametric",                  # Either "parametric" (default), "local", "mean", or "glmGamPoi" for the type of fitting of dispersions to the mean intensity
            sfType="poscounts",                    # Type of size factor estimation
            betaPrior=False,                       # Whether or not to put a zero-mean normal prior on the non-intercept coefficients (default: False)
            minReplicatesForReplace=float("inf"),  # The minimum number of replicates required in order to use replaceOutliers on a sample. 
            useT=True,                             # Default is FALSE, where Wald statistics are assumed to follow a standard Normal
            minmu=1e-06,                           # Lower bound on the estimated count for fitting gene-wise dispersion (default: 1e-06 for glmGamPoi)

        )
        dds.get_deseq_result(contrast=["sample", "C", "E"])
        res = dds.deseq_result

        # Add column with cluster name
        res["cluster"] = cluster
        
        res.to_csv(f"deseq2_mean_results/deseq2_mean_{cluster.replace(' ', '-').replace('/-', '')}.csv")
    
    except Exception as e:
        print(cluster + " errored out with error: " + str(e))

Sat 18 Jun 2022 03:37:27 PM PDT WARNING R[write to console]: estimating size factors

Sat 18 Jun 2022 03:37:27 PM PDT WARNING R[write to console]: estimating dispersions

Sat 18 Jun 2022 03:37:27 PM PDT WARNING R[write to console]: gene-wise dispersion estimates

Sat 18 Jun 2022 03:37:28 PM PDT WARNING R[write to console]: mean-dispersion relationship

Sat 18 Jun 2022 03:37:28 PM PDT WARNING R[write to console]: -- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

Sat 18 Jun 2022 03:37:28 PM PDT WARNING R[write to console]: Error in lfproc(x, y, weights = weights, cens = cens, base = base, geth = geth,  : 
  newsplit: out of vertex space

Sat 18 Jun 2022 03:37:28 PM PDT WARNING R[write to console]: In addition: 
Sat 18 Jun 2022 03:37:28 PM PDT WARNING R[write to console]: There were 25 warnings (use wa

mural cells 2 errored out with error: Error in lfproc(x, y, weights = weights, cens = cens, base = base, geth = geth,  : 
  newsplit: out of vertex space



Sat 18 Jun 2022 03:37:29 PM PDT WARNING R[write to console]: estimating size factors

Sat 18 Jun 2022 03:37:29 PM PDT WARNING R[write to console]: estimating dispersions

Sat 18 Jun 2022 03:37:29 PM PDT WARNING R[write to console]: gene-wise dispersion estimates

Sat 18 Jun 2022 03:37:30 PM PDT WARNING R[write to console]: mean-dispersion relationship

Sat 18 Jun 2022 03:37:30 PM PDT WARNING R[write to console]: -- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

Sat 18 Jun 2022 03:37:30 PM PDT WARNING R[write to console]: final dispersion estimates

Sat 18 Jun 2022 03:37:31 PM PDT WARNING R[write to console]: fitting model and testing

Sat 18 Jun 2022 03:37:32 PM PDT INFO Using contrast: ['sample', 'C', 'E']
Sat 18 Jun 2022 03:37:34 PM PDT WARNING R[write to console]: estimating size factors

Sat 18 

GABAergic neurons 1 errored out with error: Error in estimateDispersionsFit(object, fitType = fitType, quiet = quiet) : 
  all gene-wise dispersion estimates are within 2 orders of magnitude
  from the minimum value, and so the standard curve fitting techniques will not work.
  One can instead use the gene-wise estimates as final estimates:
  dds <- estimateDispersionsGeneEst(dds)
  dispersions(dds) <- mcols(dds)$dispGeneEst
  ...then continue with testing using nbinomWaldTest or nbinomLRT



Sat 18 Jun 2022 03:37:36 PM PDT WARNING R[write to console]: estimating size factors

Sat 18 Jun 2022 03:37:36 PM PDT WARNING R[write to console]: estimating dispersions

Sat 18 Jun 2022 03:37:36 PM PDT WARNING R[write to console]: gene-wise dispersion estimates

Sat 18 Jun 2022 03:37:37 PM PDT WARNING R[write to console]: mean-dispersion relationship

Sat 18 Jun 2022 03:37:37 PM PDT WARNING R[write to console]: -- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

Sat 18 Jun 2022 03:37:37 PM PDT WARNING R[write to console]: final dispersion estimates

Sat 18 Jun 2022 03:37:39 PM PDT WARNING R[write to console]: fitting model and testing

Sat 18 Jun 2022 03:37:40 PM PDT INFO Using contrast: ['sample', 'C', 'E']
Sat 18 Jun 2022 03:37:41 PM PDT WARNING R[write to console]: estimating size factors

Sat 18 

...

oligodendrocytes 2 errored out with error: Error in estimateDispersionsFit(object, fitType = fitType, quiet = quiet) : 
  all gene-wise dispersion estimates are within 2 orders of magnitude
  from the minimum value, and so the standard curve fitting techniques will not work.
  One can instead use the gene-wise estimates as final estimates:
  dds <- estimateDispersionsGeneEst(dds)
  dispersions(dds) <- mcols(dds)$dispGeneEst
  ...then continue with testing using nbinomWaldTest or nbinomLRT



Sat 18 Jun 2022 03:38:13 PM PDT WARNING R[write to console]: estimating size factors

Sat 18 Jun 2022 03:38:14 PM PDT WARNING R[write to console]: estimating dispersions

Sat 18 Jun 2022 03:38:14 PM PDT WARNING R[write to console]: gene-wise dispersion estimates

Sat 18 Jun 2022 03:38:15 PM PDT WARNING R[write to console]: mean-dispersion relationship

Sat 18 Jun 2022 03:38:15 PM PDT WARNING R[write to console]: -- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

Sat 18 Jun 2022 03:38:15 PM PDT WARNING R[write to console]: final dispersion estimates

Sat 18 Jun 2022 03:38:16 PM PDT WARNING R[write to console]: fitting model and testing

Sat 18 Jun 2022 03:38:17 PM PDT INFO Using contrast: ['sample', 'C', 'E']
Sat 18 Jun 2022 03:38:18 PM PDT WARNING R[write to console]: estimating size factors

Sat 18 

oligodendrocytes 1 errored out with error: Error in lfproc(x, y, weights = weights, cens = cens, base = base, geth = geth,  : 
  newsplit: out of vertex space



Sat 18 Jun 2022 03:38:21 PM PDT WARNING R[write to console]: estimating size factors

Sat 18 Jun 2022 03:38:21 PM PDT WARNING R[write to console]: estimating dispersions

Sat 18 Jun 2022 03:38:21 PM PDT WARNING R[write to console]: gene-wise dispersion estimates

Sat 18 Jun 2022 03:38:22 PM PDT WARNING R[write to console]: mean-dispersion relationship

Sat 18 Jun 2022 03:38:22 PM PDT WARNING R[write to console]: -- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

Sat 18 Jun 2022 03:38:22 PM PDT WARNING R[write to console]: final dispersion estimates

Sat 18 Jun 2022 03:38:23 PM PDT WARNING R[write to console]: fitting model and testing

Sat 18 Jun 2022 03:38:24 PM PDT INFO Using contrast: ['sample', 'C', 'E']
Sat 18 Jun 2022 03:38:26 PM PDT WARNING R[write to console]: estimating size factors

Sat 18 

CPU times: user 2min 33s, sys: 2min 13s, total: 4min 47s
Wall time: 1min 21s


Get a data frame with the significant p-values for all clusters:

In [124]:
i = 0
for csv in glob.glob("deseq2_mean_results/*.csv"):
    df = pd.read_csv(csv)
    if len(df[df["padj"] < 0.05]["padj"]) > 0:
        if i == 0:
            df_sig = df[df["padj"] < 0.05].copy()
            i = +1
        else:
            df_sig = df_sig.append(df[df["padj"] < 0.05], ignore_index=True)

In [125]:
df_sig

,Unnamed: 0,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,gene,cluster
0,JCHAIN_ENSTGUG00000001661.2,4.75,-18.585862,191.135443,15.587912,7.875649e-05,1.785147e-03,JCHAIN_ENSTGUG00000001661.2,microglia 2
1,_ENSTGUG00000028479.1,78.50,-6.326571,1.437911,16.014314,6.286539e-05,1.785147e-03,_ENSTGUG00000028479.1,microglia 2
2,_ENSTGUG00000028531.1,15.00,-20.132631,184.945221,22.794176,1.803113e-06,1.226117e-04,_ENSTGUG00000028531.1,microglia 2
3,_ENSTGUG00000027913.1,8.00,20.075439,250.101009,37.619242,8.599199e-10,3.207501e-07,_ENSTGUG00000027913.1,oligodendrocytes 3


In [127]:
# Clean up gene name so only Ensembl ID is retained
de_genes = []
for gene in df_sig["gene"]:
    de_genes.append(gene.split("_")[1])
    
# Pass Ensembl ID to gget info
df = gget.info(de_genes, wrap_text=True)

Sat 18 Jun 2022 03:39:41 PM PDT INFO We noticed that you may have passed a version number with your Ensembl ID.
Please note that gget info will always return information linked to the latest Ensembl ID version (see 'ensembl_id').
Sat 18 Jun 2022 03:39:48 PM PDT WARNING No reviewed UniProt results were found for Ensembl ID ENSTGUG00000001661. Returning all unreviewed results.
Sat 18 Jun 2022 03:39:52 PM PDT WARNING No reviewed UniProt results were found for Ensembl ID ENSTGUG00000028479. Returning all unreviewed results.
Sat 18 Jun 2022 03:39:55 PM PDT WARNING No reviewed UniProt results were found for Ensembl ID ENSTGUG00000028531. Returning all unreviewed results.
Sat 18 Jun 2022 03:39:58 PM PDT WARNING No reviewed UniProt results were found for Ensembl ID ENSTGUG00000027913. Returning all unreviewed results.


,ensembl_id,uniprot_id,ncbi_gene_id,species,assembly_name,primary_gene_name,ensembl_gene_name,synonyms,parent_gene,protein_names,ensembl_description,uniprot_description,ncbi_description,object_type,biotype,canonical_transcript,seq_region_name,strand,start,end,all_transcripts,transcript_biotypes,transcript_names,transcript_strands,transcript_starts,transcript_ends,all_exons,exon_starts,exon_ends,all_translations,translation_starts,translation_ends
ENSTGUG00000001661,ENSTGUG00000001661.2,"[A0A674GW10, A0A674GX51, H0YTU6]",100217545,taeniopygia_guttata,bTaeGut1_v1.p,JCHAIN,JCHAIN,"[IGJ, JCHAIN]",NaN,Uncharacterized protein,"Taeniopygia guttata joiningchain of multimeric IgA andIgM (JCHAIN), mRNA.[Source:RefSeqmRNA;Acc:NM_001309502]",NaN,NaN,Gene,protein_coding,ENSTGUT00000027687.1,4,-1,67313744,67318861,"[ENSTGUT00000001728.2, ENSTGUT00000039414.1, ENSTGUT00000027687.1]","[protein_coding, protein_coding, protein_coding]","[JCHAIN-201, JCHAIN-202, JCHAIN-203]","[-1, -1, -1]","[67313744, 67313745, 67314471]","[67318861, 67318767, 67316295]",NaN,NaN,NaN,NaN,NaN,NaN
ENSTGUG00000028479,ENSTGUG00000028479.1,"[A0A674GB31, A0A674GBM2, A0A674GCD7, A0A674GHK5, A0A674GNN0, A0A674H7B8, A0A674HBQ1, A0A674HFI2, H0ZF58]",NaN,taeniopygia_guttata,bTaeGut1_v1.p,"[nan, nan, nan, nan, nan, nan, nan, nan, nan]",NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan]",NaN,"[Ig-like domain-containing protein, Uncharacterized protein]",NaN,NaN,NaN,Gene,protein_coding,ENSTGUT00000029820.1,15,1,5648389,5671784,"[ENSTGUT00000029820.1, ENSTGUT00000028177.1, ENSTGUT00000031159.1, ENSTGUT00000022357.1, ENSTGUT00000026496.1, ENSTGUT00000020849.1, ENSTGUT00000035004.1, ENSTGUT00000023157.1, ENSTGUT00000009319.2]","[protein_coding, protein_coding, protein_coding, protein_coding, protein_coding, protein_coding, protein_coding, protein_coding, protein_coding]","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[5648389, 5648389, 5652286, 5652286, 5652286, 5656220, 5656675, 5657506, 5666662]","[5671784, 5671784, 5671784, 5671784, 5671784, 5671784, 5671784, 5663165, 5671784]",NaN,NaN,NaN,NaN,NaN,NaN
ENSTGUG00000028531,ENSTGUG00000028531.1,A0A674H0A8,NaN,taeniopygia_guttata,bTaeGut1_v1.p,NaN,NaN,NaN,NaN,Ig-like domain-containing protein,NaN,NaN,NaN,Gene,protein_coding,ENSTGUT00000025236.1,RRCB01000107.1,1,976755,1081424,[ENSTGUT00000025236.1],[protein_coding],[nan],[1],[976755],[1081424],NaN,NaN,NaN,NaN,NaN,NaN
ENSTGUG00000027913,ENSTGUG00000027913.1,A0A674HVK1,NaN,taeniopygia_guttata,bTaeGut1_v1.p,NaN,NaN,NaN,NaN,Aldo_ket_red domain-containing protein,NaN,NaN,NaN,Gene,protein_coding,ENSTGUT00000024158.1,RRCB01000090.1,1,2120170,2139914,[ENSTGUT00000024158.1],[protein_coding],[nan],[1],[2120170],[2139914],NaN,NaN,NaN,NaN,NaN,NaN


Using the means with DESeq2 did not work very well, I only got 4 DE genes in total. I think part of the problem is that DESeq2 expects raw data and therefore integers, so I had to round the mean values to the closest integer (often 0). Some celltypes did not run at all because all gene-wise dispersion estimates were within 2 orders of magnitude.

___

# DESeq2 on individual cells as replicates - build loop components

Generate DESeq2 count matrix from raw counts (DESeq2 only allows integer values). Each cell will be treated as a replicate:

In [24]:
# Create masks to obtain only retain cells from control or experiment, and one cluster
celltype_mask_control = np.char.startswith(nd(adata.obs.celltype.values).astype(str), "microglia 1") & np.logical_or(adata.obs["batch"]=="control1", adata.obs["batch"]=="control2")
celltype_mask_experiment = np.char.startswith(nd(adata.obs.celltype.values).astype(str), "microglia 1") & np.logical_or(adata.obs["batch"]=="experiment1", adata.obs["batch"]=="experiment2")

In [67]:
# Apply masks to raw data
control_data = adata.raw[celltype_mask_control]
experiment_data = adata.raw[celltype_mask_experiment]

In [69]:
# Filter for only highly variable genes
control_data = control_data[:,adata.var['highly_variable']]
experiment_data = experiment_data[:,adata.var['highly_variable']]

In [73]:
subsampling_runs = 10
subsample_size = 100

count_matrix = pd.DataFrame()
count_matrix["gene"] = adata.raw.var[adata.var['highly_variable']==True].index.values

## Get expression of all genes for each cell from both control datasets
count_matrix_control = pd.DataFrame(control_data.X.todense().T.astype(int))
# Randombly subsample contol data
count_matrix_control = count_matrix_control.sample(n=subsample_size, axis="columns")
# Relabel columns 
count_matrix_control.columns = np.arange(100)+1
# Add prefix to mark these cells as control
count_matrix_control = count_matrix_control.add_prefix("C_")

## Get expression of all genes for each cell from both experiment datasets
count_matrix_experiment = pd.DataFrame(experiment_data.X.todense().T.astype(int))
# Randombly subsample experiment data
count_matrix_experiment = count_matrix_experiment.sample(n=subsample_size, axis="columns")
# Relabel columns 
count_matrix_experiment.columns = np.arange(100)+1
# Add prefix to mark these cells as experiment
count_matrix_experiment = count_matrix_experiment.add_prefix("E_")

# Concatenate experiment and control data
count_matrix = pd.concat(
    [count_matrix, count_matrix_control, count_matrix_experiment], axis=1
)
count_matrix

,gene,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,...,E_91,E_92,E_93,E_94,E_95,E_96,E_97,E_98,E_99,E_100
0,WRE_WRE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,TMEM74_ENSTGUG00000022792.1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,TMEM170B_ENSTGUG00000025839.1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,_ENSTGUG00000020832.1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,_ENSTGUG00000020221.1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4479,_ENSTGUG00000021313.1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4480,KIF5A_ENSTGUG00000025926.1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4481,B4GALNT1_ENSTGUG00000028363.1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4482,AGAP2_ENSTGUG00000019988.1,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0


Generate DESeq2 design matrix:

In [74]:
design_matrix = pd.DataFrame()
design_matrix["samplename"] = count_matrix.columns.values[1:]
design_matrix = design_matrix.assign(
    sample=lambda d: d.samplename.str.extract("([CE])_", expand=False)
)
design_matrix["replicate"] = design_matrix["samplename"].str.extractall("(\d+)").values
design_matrix.index = design_matrix.samplename
design_matrix

,samplename,sample,replicate
samplename,,,
C_1,C_1,C,1
C_2,C_2,C,2
C_3,C_3,C,3
C_4,C_4,C,4
C_5,C_5,C,5
...,...,...,...
E_96,E_96,E,96
E_97,E_97,E,97
E_98,E_98,E,98


Run DESeq2:

In [85]:
%%time
dds = py_DESeq2(
    count_matrix=count_matrix,
    design_matrix=design_matrix,
    design_formula="~ replicate + sample",
    gene_column="gene",
)
dds.run_deseq(
    test="LRT",                            # Likelihood ratio test on the difference in deviance between a full and reduced model formula (defined by nbinomLRT)
    reduced="~ replicate",                 # Reduced model without condition as variable
    # fitType="parametric",                # Either "parametric", "local", "mean", or "glmGamPoi" for the type of fitting of dispersions to the mean intensity
    sfType="poscounts",                    # Type of size factor estimation
    betaPrior=False,                       # Whether or not to put a zero-mean normal prior on the non-intercept coefficients (default: False)
    minReplicatesForReplace=float("inf"),  # The minimum number of replicates required in order to use replaceOutliers on a sample. 
    useT=True,                             # Default is FALSE, where Wald statistics are assumed to follow a standard Normal
    minmu=1e-06,                           # Lower bound on the estimated count for fitting gene-wise dispersion (default: 1e-06 for glmGamPoi)
 
)
dds.get_deseq_result(contrast=["sample", "C", "E"])
res = dds.deseq_result

Thu 16 Jun 2022 01:48:37 PM PDT WARNING R[write to console]: estimating size factors

Thu 16 Jun 2022 01:48:37 PM PDT WARNING R[write to console]: estimating dispersions

Thu 16 Jun 2022 01:48:37 PM PDT WARNING R[write to console]: gene-wise dispersion estimates

Thu 16 Jun 2022 01:51:43 PM PDT WARNING R[write to console]: mean-dispersion relationship

Thu 16 Jun 2022 01:51:43 PM PDT WARNING R[write to console]: -- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

Thu 16 Jun 2022 01:51:43 PM PDT WARNING R[write to console]: final dispersion estimates

Thu 16 Jun 2022 02:00:43 PM PDT WARNING R[write to console]: fitting model and testing

Thu 16 Jun 2022 02:09:48 PM PDT INFO Using contrast: ['sample', 'C', 'E']


CPU times: user 8h 23min 4s, sys: 8h 25min 37s, total: 16h 48min 41s
Wall time: 21min 16s


In [86]:
res

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,gene
WRE_WRE,0.010030,11.700454,59.028487,0.096264,0.756360,1.0,WRE_WRE
TMEM74_ENSTGUG00000022792.1,0.005026,10.716314,62.629328,0.047275,0.827874,1.0,TMEM74_ENSTGUG00000022792.1
TMEM170B_ENSTGUG00000025839.1,0.025015,-5.751915,13.670826,0.027882,0.867387,1.0,TMEM170B_ENSTGUG00000025839.1
_ENSTGUG00000020832.1,0.010034,11.703627,59.019681,0.096315,0.756298,1.0,_ENSTGUG00000020832.1
_ENSTGUG00000020221.1,0.060168,-4.831429,9.907635,0.023088,0.879230,1.0,_ENSTGUG00000020221.1
...,...,...,...,...,...,...,...
_ENSTGUG00000021313.1,0.015038,12.269741,57.479802,0.145430,0.702941,1.0,_ENSTGUG00000021313.1
KIF5A_ENSTGUG00000025926.1,0.000000,NaN,NaN,NaN,NaN,NaN,KIF5A_ENSTGUG00000025926.1
B4GALNT1_ENSTGUG00000028363.1,0.000000,NaN,NaN,NaN,NaN,NaN,B4GALNT1_ENSTGUG00000028363.1
AGAP2_ENSTGUG00000019988.1,0.035050,-9.238563,17.473478,0.131692,0.716684,1.0,AGAP2_ENSTGUG00000019988.1


In [87]:
# Show significant genes
res[res["padj"]<0.05]

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,gene
_ENSTGUG00000004607.2,2.686180,-1.457530,0.315486,15.346645,8.947966e-05,0.048006,_ENSTGUG00000004607.2
CD74_ENSTGUG00000000882.2,15.337836,-0.807027,0.149539,24.534563,7.298874e-07,0.000470,CD74_ENSTGUG00000000882.2
_ENSTGUG00000028479.1,12.372307,-17.922297,14.393935,3427.974502,0.000000e+00,0.000000,_ENSTGUG00000028479.1
_ENSTGUG00000019883.1,11.579027,-1.210667,0.183098,34.744482,3.759428e-09,0.000004,_ENSTGUG00000019883.1
_ENSTGUG00000029343.1,9.966743,-17.773859,15.224227,2761.491057,0.000000e+00,0.000000,_ENSTGUG00000029343.1
_ENSTGUG00000023468.1,3.142643,-1.383097,0.227779,31.968212,1.567162e-08,0.000013,_ENSTGUG00000023468.1


In [88]:
# res.to_csv("deseq2_micro_test.csv")

In [89]:
# Show normalized gene counts
# dds.normalized_count()

Show coefficients for GLM:

In [80]:
# dds.comparison

In [164]:
# # from the last cell, we see the arrangement of coefficients, 
# # so that we can now use "coef" for lfcShrink
# # the comparison we want to focus on is 'sample_E_vs_C', so coef = 195 will be used
# lfc_res = dds.lfcShrink(coef=195, method='apeglm')
# lfc_res.to_csv("deseq2_gaba_subsample_lfcShrink.csv")
# lfc_res.head()

___

# DESeq2 with individual cells as replicates - Loop over all clusters

In [113]:
# Show cluster sizes
adata.obs.celltype.value_counts()

glutamatergic neurons 1                 4144
GABAergic neurons 1                     4084
migrating neuroblasts                   3336
microglia 1                             3039
glutamatergic neurons 2                 2846
radial glia 1                           2449
mural / vascular endothelial cells 1    2377
astrocytes 1                            2127
glutamatergic neurons 3                 1990
microglia 2                             1924
astrocytes 2                            1695
oligodendrocytes 1                      1570
GABAergic neurons 2                     1508
mural cells 2                            821
oligodendrocyte precursor cells          474
radial glia 2                            407
red blood cells                          359
glutamatergic neurons 4                  296
oligodendrocytes 2                       213
oligodendrocytes 3                        61
GABAergic neurons 3                       59
microglia 3 / radial glia                 25
Name: cell

In [ ]:
# %%time
# # Define number of subsampling runs
# subsampling_runs = 10

# for cluster in adata.obs.celltype.values.unique():   
#     # Define subsample size
#     cluster_size = len(adata.obs[adata.obs["celltype"]==cluster])
#     # If cluster includes > 400 cells, subsample size = 100 cells per condition
#     if cluster_size >= 400:
#         subsample_size = 100
#     # If cluster includes < 400 cells, do not subsample
#     elif cluster_size < 400 and cluster_size > 100:
#         subsample_size = False
#     # Exclude clusters with < 100 cells
#     else:
#         continue
    
#     print(f"Running DESeq2 on celltype cluster {cluster} with subsample size {subsample_size}.")
        
#     for i in np.arange(subsampling_runs):
#         # Create masks to only retain cells from control or experiment, and one cluster
#         celltype_mask_control = np.char.startswith(nd(adata.obs.celltype.values).astype(str), cluster) & np.logical_or(adata.obs["batch"]=="control1", adata.obs["batch"]=="control2")
#         celltype_mask_experiment = np.char.startswith(nd(adata.obs.celltype.values).astype(str), cluster) & np.logical_or(adata.obs["batch"]=="experiment1", adata.obs["batch"]=="experiment2")

#         # Apply masks to raw data
#         control_data = adata.raw[celltype_mask_control]
#         experiment_data = adata.raw[celltype_mask_experiment]

#         # Filter for only highly variable genes
#         control_data = control_data[:,adata.var['highly_variable']]
#         experiment_data = experiment_data[:,adata.var['highly_variable']]

#         ### Create count matrix
#         count_matrix = pd.DataFrame()
#         count_matrix["gene"] = adata.raw.var[adata.var['highly_variable']==True].index.values

#         ## Create matrix including raw expression of all highly variable genes for each cell from both control animals
#         count_matrix_control = pd.DataFrame(control_data.X.todense().T.astype(int))
        
#         if subsample_size:
#             # Randombly subsample contol data
#             count_matrix_control = count_matrix_control.sample(n=subsample_size, axis="columns")
#             # Relabel columns 
#             count_matrix_control.columns = np.arange(subsample_size)+1
            
#         # Add prefix to mark these cells as control
#         count_matrix_control = count_matrix_control.add_prefix("C_")

#         ## Get raw expression of all highly variable genes for each cell from both experiment animals
#         count_matrix_experiment = pd.DataFrame(experiment_data.X.todense().T.astype(int))
        
#         if subsample_size:
#             # Randombly subsample experiment data
#             count_matrix_experiment = count_matrix_experiment.sample(n=subsample_size, axis="columns")
#             # Relabel columns 
#             count_matrix_experiment.columns = np.arange(subsample_size)+1
            
#         # Add prefix to mark these cells as experiment
#         count_matrix_experiment = count_matrix_experiment.add_prefix("E_")

#         ## Concatenate experiment and control data
#         count_matrix = pd.concat(
#             [count_matrix, count_matrix_control, count_matrix_experiment], axis=1
#         )

#         # Show the first count matrix for this cluster
#         if i==0:
#             print(count_matrix)

#         ### Create design matrix
#         design_matrix = pd.DataFrame()
#         design_matrix["samplename"] = count_matrix.columns.values[1:]
#         # Get sample ID and replicate number from count matrix column names
#         design_matrix = design_matrix.assign(
#             sample=lambda d: d.samplename.str.extract("([CE])_", expand=False)
#         )
#         design_matrix["replicate"] = design_matrix["samplename"].str.extractall("(\d+)").values
#         # Set sample name as index
#         design_matrix.index = design_matrix.samplename

#         # Show the first design matrix for this cluster
#         if i==0:
#             print(design_matrix)

#         ### Run DESeq2
#         dds = py_DESeq2(
#             count_matrix=count_matrix,
#             design_matrix=design_matrix,
#             design_formula="~ replicate + sample",
#             gene_column="gene",
#         )
#         dds.run_deseq(
#             test="LRT",                            # Likelihood ratio test on the difference in deviance between a full and reduced model formula (defined by nbinomLRT)
#             reduced="~ replicate",                 # Reduced model without condition as variable
#             # fitType="parametric",                # Either "parametric", "local", "mean", or "glmGamPoi" for the type of fitting of dispersions to the mean intensity (default: parametric)
#             sfType="poscounts",                    # Type of size factor estimation
#             betaPrior=False,                       # Whether or not to put a zero-mean normal prior on the non-intercept coefficients (default: False)
#             minReplicatesForReplace=float("inf"),  # The minimum number of replicates required in order to use replaceOutliers on a sample. 
#             useT=True,                             # Default is FALSE, where Wald statistics are assumed to follow a standard Normal
#             minmu=1e-06,                           # Lower bound on the estimated count for fitting gene-wise dispersion (default: 1e-06 for glmGamPoi)

#         )
#         dds.get_deseq_result(contrast=["sample", "C", "E"])
#         res = dds.deseq_result
        
#         res.to_csv(f"deseq2_results/deseq2_{cluster.replace(' ', '-').replace('/-', '')}_subsample_{i}.csv") # Results folder renamed to deseq2_results_v1
        
#         # Do not run again if cluster was not subsampled
#         if subsample_size is False:
#             break

### Redo with equal sampling from each replicate (deseq2_results_v2)

In [15]:
celltypes = [
 'red blood cells',
 'mural / vascular endothelial cells 1',
 'glutamatergic neurons 3',
 'glutamatergic neurons 1',
 'migrating neuroblasts',
 'GABAergic neurons 2',
 'glutamatergic neurons 2',
 'radial glia 1',
 'oligodendrocyte precursor cells',
 'oligodendrocytes 2',
 'microglia 2',
 'oligodendrocytes 1',
 'radial glia 2',
 'glutamatergic neurons 4',
 'oligodendrocytes 3',
 'GABAergic neurons 3',
 'microglia 3 / radial glia']

In [20]:
%%time
# Define number of subsampling runs
subsampling_runs = 3

# for cluster in adata.obs.celltype.values.unique():   
for cluster in celltypes:  
    # Define subsample size (per condition; split 50/50 across replicates)
    cluster_size = len(adata.obs[adata.obs["celltype"]==cluster])
    # If cluster includes > 500 cells, subsample size = 100 cells per condition
    if cluster_size >= 500:
        subsample_size = 100
    # If cluster includes < 500 cells, do not subsample
    elif cluster_size < 500 and cluster_size >= 100:
        subsample_size = False
    # Exclude clusters with < 100 cells
    else:
        continue
    
    print(f"Running DESeq2 on celltype cluster {cluster} with subsample size {subsample_size}.")
        
    for i in np.arange(subsampling_runs):
        ### Get control count data
        ## Create mask to only retain cells from control 1, and one cluster
        celltype_mask_control1 = np.char.startswith(nd(adata.obs.celltype.values).astype(str), cluster) & [adata.obs["batch"]=="control1"]
        control_data1 = adata.raw[celltype_mask_control1]
        
        # Filter for only highly variable genes
        control_data1 = control_data1[:,adata.var['highly_variable']]
        
        count_matrix_control1 = pd.DataFrame(control_data1.X.todense().T.astype(int))

        if subsample_size:
            # Randombly subsample contol 1 data
            count_matrix_control1 = count_matrix_control1.sample(n=int(subsample_size/2), axis="columns")
            # Renumber columns 
            count_matrix_control1.columns = np.arange(int(subsample_size/2))+1
        else:
            # Relabel columns with indexing from 1 instead of 0
            count_matrix_control1.columns = count_matrix_control1.columns+1

        ## Create mask to only retain cells from control 2, and one cluster
        celltype_mask_control2 = np.char.startswith(nd(adata.obs.celltype.values).astype(str), cluster) & [adata.obs["batch"]=="control2"]
        control_data2 = adata.raw[celltype_mask_control2]
        
        # Filter for only highly variable genes
        control_data2 = control_data2[:,adata.var['highly_variable']]
        
        count_matrix_control2 = pd.DataFrame(control_data2.X.todense().T.astype(int))

        if subsample_size:
            # Randombly subsample contol 2 data
            count_matrix_control2 = count_matrix_control2.sample(n=int(subsample_size/2), axis="columns")
            # Renumber columns 
            count_matrix_control2.columns = np.arange(int(subsample_size/2))+1+(int(subsample_size/2))
        else:
            # Relabel columns with indexing from 1 instead of 0 + max replicate number from previous replicate
            count_matrix_control2.columns = count_matrix_control2.columns+1+max(count_matrix_control1.columns)

        # Add prefix to mark these cells as control
        count_matrix_control1 = count_matrix_control1.add_prefix("C_")
        count_matrix_control2 = count_matrix_control2.add_prefix("C_")
        
        ### Get experiment count data
        ## Create mask to only retain cells from experiment 1, and one cluster
        celltype_mask_experiment1 = np.char.startswith(nd(adata.obs.celltype.values).astype(str), cluster) & [adata.obs["batch"]=="experiment1"]
        experiment_data1 = adata.raw[celltype_mask_experiment1]
        
        # Filter for only highly variable genes
        experiment_data1 = experiment_data1[:,adata.var['highly_variable']]
        
        count_matrix_experiment1 = pd.DataFrame(experiment_data1.X.todense().T.astype(int))

        if subsample_size:
            # Randombly subsample contol 1 data
            count_matrix_experiment1 = count_matrix_experiment1.sample(n=int(subsample_size/2), axis="columns")
            # Renumber columns 
            count_matrix_experiment1.columns = np.arange(int(subsample_size/2))+1
        else:
            # Relabel columns with indexing from 1 instead of 0
            count_matrix_experiment1.columns = count_matrix_experiment1.columns+1

        ## Create mask to only retain cells from experiment 2, and one cluster
        celltype_mask_experiment2 = np.char.startswith(nd(adata.obs.celltype.values).astype(str), cluster) & [adata.obs["batch"]=="experiment2"]
        experiment_data2 = adata.raw[celltype_mask_experiment2]
        
        # Filter for only highly variable genes
        experiment_data2 = experiment_data2[:,adata.var['highly_variable']]
        
        count_matrix_experiment2 = pd.DataFrame(experiment_data2.X.todense().T.astype(int))

        if subsample_size:
            # Randombly subsample contol 1 data
            count_matrix_experiment2 = count_matrix_experiment2.sample(n=int(subsample_size/2), axis="columns")
            # Renumber columns 
            count_matrix_experiment2.columns = np.arange(int(subsample_size/2))+1+(int(subsample_size/2))
        else:
            # Relabel columns with indexing from 1 instead of 0 + max replicate number from previous replicate
            count_matrix_experiment2.columns = count_matrix_experiment2.columns+1+max(count_matrix_experiment1.columns)

        # Add prefix to mark these cells as experiment
        count_matrix_experiment1 = count_matrix_experiment1.add_prefix("E_")
        count_matrix_experiment2 = count_matrix_experiment2.add_prefix("E_")


        ## Concatenate experiment and control data
        count_matrix = pd.DataFrame()
        count_matrix["gene"] = adata.raw.var[adata.var['highly_variable']==True].index.values
        count_matrix = pd.concat(
            [count_matrix, count_matrix_control1, count_matrix_control2, count_matrix_experiment1, count_matrix_experiment2], axis=1
        )

        # Show the first count matrix for this cluster
        if i==0:
            print(count_matrix)

        ### Create design matrix
        design_matrix = pd.DataFrame()
        design_matrix["samplename"] = count_matrix.columns.values[1:]
        # Get sample ID and replicate number from count matrix column names
        design_matrix = design_matrix.assign(
            sample=lambda d: d.samplename.str.extract("([CE])_", expand=False)
        )
        design_matrix["replicate"] = design_matrix["samplename"].str.extractall("(\d+)").values
        # Set sample name as index
        design_matrix.index = design_matrix.samplename

        # Show the first design matrix for this cluster
        if i==0:
            print(design_matrix)

        ### Run DESeq2
        dds = py_DESeq2(
            count_matrix=count_matrix,
            design_matrix=design_matrix,
            design_formula="~ replicate + sample",
            gene_column="gene",
        )
        dds.run_deseq(
            test="LRT",                            # Likelihood ratio test on the difference in deviance between a full and reduced model formula (defined by nbinomLRT)
            reduced="~ replicate",                 # Reduced model without condition as variable
            # fitType="parametric",                # Either "parametric", "local", "mean", or "glmGamPoi" for the type of fitting of dispersions to the mean intensity (default: parametric)
            sfType="poscounts",                    # Type of size factor estimation
            betaPrior=False,                       # Whether or not to put a zero-mean normal prior on the non-intercept coefficients (default: False)
            minReplicatesForReplace=float("inf"),  # The minimum number of replicates required in order to use replaceOutliers on a sample. 
            useT=True,                             # Default is FALSE, where Wald statistics are assumed to follow a standard Normal
            minmu=1e-06,                           # Lower bound on the estimated count for fitting gene-wise dispersion (default: 1e-06 for glmGamPoi)

        )
        dds.get_deseq_result(contrast=["sample", "C", "E"])
        res = dds.deseq_result
        
        res.to_csv(f"deseq2_results_v2/deseq2_{cluster.replace(' ', '-').replace('/-', '')}_subsample_{i}.csv")
        
        # Do not run again if cluster was not subsampled
        if subsample_size is False:
            break

Running DESeq2 on celltype cluster red blood cells with subsample size False.
                               gene  C_1  C_2  C_3  C_4  C_5  C_6  C_7  C_8  \
0                           WRE_WRE    0    0    0    0    0    0    0    2   
1       TMEM74_ENSTGUG00000022792.1    0    0    0    0    0    0    0    0   
2     TMEM170B_ENSTGUG00000025839.1    0    0    0    0    0    0    0    0   
3             _ENSTGUG00000020832.1    0    0    0    0    0    0    0    0   
4             _ENSTGUG00000020221.1    0    0    0    0    0    0    0    0   
...                             ...  ...  ...  ...  ...  ...  ...  ...  ...   
4479          _ENSTGUG00000021313.1    0    0    0    0    0    0    0    0   
4480     KIF5A_ENSTGUG00000025926.1    0    0    0    0    0    0    0    0   
4481  B4GALNT1_ENSTGUG00000028363.1    0    0    0    0    0    0    0    0   
4482     AGAP2_ENSTGUG00000019988.1    0    0    0    0    0    0    0    0   
4483          _ENSTGUG00000022543.1    1    0    0   

Wed 22 Jun 2022 02:21:45 PM PDT WARNING R[write to console]: estimating size factors

Wed 22 Jun 2022 02:21:46 PM PDT WARNING R[write to console]: estimating dispersions

Wed 22 Jun 2022 02:21:46 PM PDT WARNING R[write to console]: gene-wise dispersion estimates

Wed 22 Jun 2022 02:33:49 PM PDT WARNING R[write to console]: mean-dispersion relationship

Wed 22 Jun 2022 02:33:49 PM PDT WARNING R[write to console]: final dispersion estimates

Wed 22 Jun 2022 03:04:00 PM PDT WARNING R[write to console]: fitting model and testing

Wed 22 Jun 2022 03:35:37 PM PDT INFO Using contrast: ['sample', 'C', 'E']


Running DESeq2 on celltype cluster mural / vascular endothelial cells 1 with subsample size 100.
                               gene  C_1  C_2  C_3  C_4  C_5  C_6  C_7  C_8  \
0                           WRE_WRE    0    0    0    0    0    0    0    0   
1       TMEM74_ENSTGUG00000022792.1    0    0    0    0    0    0    0    0   
2     TMEM170B_ENSTGUG00000025839.1    0    0    0    0    0    0    0    0   
3             _ENSTGUG00000020832.1    0    0    0    0    0    0    0    0   
4             _ENSTGUG00000020221.1    0    0    0    0    0    0    0    0   
...                             ...  ...  ...  ...  ...  ...  ...  ...  ...   
4479          _ENSTGUG00000021313.1    0    0    0    0    0    0    0    0   
4480     KIF5A_ENSTGUG00000025926.1    0    0    0    0    0    0    0    0   
4481  B4GALNT1_ENSTGUG00000028363.1    0    0    0    0    0    0    0    0   
4482     AGAP2_ENSTGUG00000019988.1    0    0    0    0    0    0    0    0   
4483          _ENSTGUG00000022543.

Wed 22 Jun 2022 03:35:43 PM PDT WARNING R[write to console]: estimating size factors

Wed 22 Jun 2022 03:35:43 PM PDT WARNING R[write to console]: estimating dispersions

Wed 22 Jun 2022 03:35:43 PM PDT WARNING R[write to console]: gene-wise dispersion estimates

Wed 22 Jun 2022 03:38:48 PM PDT WARNING R[write to console]: mean-dispersion relationship

Wed 22 Jun 2022 03:38:48 PM PDT WARNING R[write to console]: -- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

Wed 22 Jun 2022 03:38:49 PM PDT WARNING R[write to console]: final dispersion estimates

Wed 22 Jun 2022 03:47:28 PM PDT WARNING R[write to console]: fitting model and testing

Wed 22 Jun 2022 03:56:45 PM PDT WARNING R[write to console]: 1 rows did not converge in beta, labelled in mcols(object)$fullBetaConv. Use larger maxit argument with nb

Running DESeq2 on celltype cluster glutamatergic neurons 3 with subsample size 100.
                               gene  C_1  C_2  C_3  C_4  C_5  C_6  C_7  C_8  \
0                           WRE_WRE    0    0    0    0    0    0    0    0   
1       TMEM74_ENSTGUG00000022792.1    0    0    0    0    0    0    0    0   
2     TMEM170B_ENSTGUG00000025839.1    0    0    0    0    0    0    0    0   
3             _ENSTGUG00000020832.1    0    0    0    0    0    0    0    0   
4             _ENSTGUG00000020221.1    0    0    0    0    0    0    0    2   
...                             ...  ...  ...  ...  ...  ...  ...  ...  ...   
4479          _ENSTGUG00000021313.1    0    0    0    0    1    0    0    0   
4480     KIF5A_ENSTGUG00000025926.1    0    0    0    0    0    0    0    0   
4481  B4GALNT1_ENSTGUG00000028363.1    0    0    0    0    0    0    0    0   
4482     AGAP2_ENSTGUG00000019988.1    0    0    2    0    0    1    0    0   
4483          _ENSTGUG00000022543.1    1    0  

...

Running DESeq2 on celltype cluster radial glia 2 with subsample size False.
                               gene  C_1  C_2  C_3  C_4  C_5  C_6  C_7  C_8  \
0                           WRE_WRE    0    0    0    0    0    0    0    0   
1       TMEM74_ENSTGUG00000022792.1    0    0    0    0    0    0    0    0   
2     TMEM170B_ENSTGUG00000025839.1    0    0    0    0    0    0    0    0   
3             _ENSTGUG00000020832.1    4    0    0    0    0    0    0    0   
4             _ENSTGUG00000020221.1    1    0    0    1    3    0    1    0   
...                             ...  ...  ...  ...  ...  ...  ...  ...  ...   
4479          _ENSTGUG00000021313.1    0    0    0    0    0    0    0    0   
4480     KIF5A_ENSTGUG00000025926.1    0    0    0    0    0    0    0    0   
4481  B4GALNT1_ENSTGUG00000028363.1    0    0    0    0    0    0    0    0   
4482     AGAP2_ENSTGUG00000019988.1    0    0    0    1    0    0    0    0   
4483          _ENSTGUG00000022543.1    0    0    0    0

Thu 23 Jun 2022 04:27:30 AM PDT WARNING R[write to console]: estimating size factors

Thu 23 Jun 2022 04:27:31 AM PDT WARNING R[write to console]: estimating dispersions

Thu 23 Jun 2022 04:27:31 AM PDT WARNING R[write to console]: gene-wise dispersion estimates

Thu 23 Jun 2022 04:46:14 AM PDT WARNING R[write to console]: mean-dispersion relationship

Thu 23 Jun 2022 04:46:14 AM PDT WARNING R[write to console]: final dispersion estimates

Thu 23 Jun 2022 05:28:33 AM PDT WARNING R[write to console]: fitting model and testing

Thu 23 Jun 2022 06:23:17 AM PDT INFO Using contrast: ['sample', 'C', 'E']


Running DESeq2 on celltype cluster glutamatergic neurons 4 with subsample size False.
                               gene  C_1  C_2  C_3  C_4  C_5  C_6  C_7  C_8  \
0                           WRE_WRE    0    2    1    0    5    0    0    5   
1       TMEM74_ENSTGUG00000022792.1    0    1    0    0    1    0    0    0   
2     TMEM170B_ENSTGUG00000025839.1    0    2    2    0    0    0    0    0   
3             _ENSTGUG00000020832.1    0    0    0    0    0    0    0    0   
4             _ENSTGUG00000020221.1    0    1    0    2    1    2    0    0   
...                             ...  ...  ...  ...  ...  ...  ...  ...  ...   
4479          _ENSTGUG00000021313.1    0    1    0    0    0    1    0    0   
4480     KIF5A_ENSTGUG00000025926.1    0    0    0    2    0    0    0    0   
4481  B4GALNT1_ENSTGUG00000028363.1    0    0    0    0    0    0    0    0   
4482     AGAP2_ENSTGUG00000019988.1    4    0    2    8    0    3    1    0   
4483          _ENSTGUG00000022543.1    0    0

Thu 23 Jun 2022 06:23:25 AM PDT WARNING R[write to console]: estimating size factors

Thu 23 Jun 2022 06:23:26 AM PDT WARNING R[write to console]: estimating dispersions

Thu 23 Jun 2022 06:23:26 AM PDT WARNING R[write to console]: gene-wise dispersion estimates

Thu 23 Jun 2022 06:37:18 AM PDT WARNING R[write to console]: mean-dispersion relationship

Thu 23 Jun 2022 06:37:18 AM PDT WARNING R[write to console]: -- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

Thu 23 Jun 2022 06:37:18 AM PDT WARNING R[write to console]: final dispersion estimates

Thu 23 Jun 2022 07:03:06 AM PDT WARNING R[write to console]: fitting model and testing

Thu 23 Jun 2022 07:37:04 AM PDT INFO Using contrast: ['sample', 'C', 'E']


CPU times: user 17d 1h 31min 35s, sys: 16d 17h 51min 1s, total: 33d 19h 22min 37s
Wall time: 17h 15min 29s


___

For each cluster, keep only the genes that are significant across all subsamples:

In [21]:
i = 0
for cluster in adata.obs.celltype.values.unique():
    genes_to_keep = []
    
    # Get all the csv files for this cluster
    for csv in glob.glob(f"deseq2_results_v2/deseq2_{cluster.replace(' ', '-').replace('/-', '')}_subsample_*.csv"):
        df = pd.read_csv(csv)
        
        # Bonferroni correct alpha for # subsamples x number of (highly variable) genes
        number_of_subsamples = len(glob.glob(f"deseq2_results_v2/deseq2_{cluster.replace(' ', '-').replace('/-', '')}_subsample_*.csv"))
        alpha = 0.05 / (number_of_subsamples * len(adata.var[adata.var['highly_variable']==True]))
        
        if len(df[df["padj"] < alpha]["padj"]) > 0:
            if i == 0:
                # For the first iteration, set genes_to_keep equal to the significant genes from this subsample
                genes_to_keep = df[df["padj"] < alpha]["gene"].values
                i = +1
            else:
                # Only keep genes that are also significant in this subsample
                genes_to_keep = list(set(genes_to_keep) & set(df[df["padj"] < alpha]["gene"].values))
                
        # If no genes are significant: empty genes to keep, break loop and move to next cluster
        else:
            genes_to_keep = []
            break
    
    if len(genes_to_keep) > 0:
        df_sig = pd.DataFrame()
        df_sig["de_gene"] = genes_to_keep
        df_sig["cluster"] = cluster
        df_sig.to_csv(f"deseq2_results_v2/significant_across_all/deseq2_{cluster.replace(' ', '-').replace('/-', '')}.csv")

Keep all genes that are significant in at least one subsample:

In [22]:
i = 0
for cluster in adata.obs.celltype.values.unique():
    df_sig = pd.DataFrame()
    
    # Get all the csv files for this cluster
    for csv in glob.glob(f"deseq2_results_v2/deseq2_{cluster.replace(' ', '-').replace('/-', '')}_subsample_*.csv"):
        df = pd.read_csv(csv)
        # Add cluster name
        df["cluster"] = cluster
        
        # Bonferroni correct alpha for # subsamples x number of (highly variable) genes
        number_of_subsamples = len(glob.glob(f"deseq2_results_v2/deseq2_{cluster.replace(' ', '-').replace('/-', '')}_subsample_*.csv"))
        alpha = 0.05 / (number_of_subsamples * len(adata.var[adata.var['highly_variable']==True]))
        
        # Add number of subsamples to df
        df["number_of_subsamples"] = number_of_subsamples
        
        if len(df[df["padj"] < alpha]["padj"]) > 0:
            if i == 0:
                df_sig = df[df["padj"] < alpha].copy()
                i = +1
            else:
                df_sig = df_sig.append(df[df["padj"] < alpha], ignore_index=True)
    
    if len(df_sig) > 0:
        df_sig.to_csv(f"deseq2_results_v2/significant/deseq2_{cluster.replace(' ', '-').replace('/-', '')}.csv")